<h1 align=center><font size = 5>IBM Applied data science capstone</font></h1>

![](scottishFlag.png)

In [81]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import folium
from bs4 import BeautifulSoup
import requests
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [83]:
req = requests.get("https://en.wikipedia.org/wiki/EH_postcode_area")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[1]

df_temp = pd.read_html(str(table))

df_edin = pd.DataFrame(df_temp[0])

In [84]:
df_edin.columns = ['Postal Code', 'Borough', 'Neighborhood', 'Local area']
df_edin = df_edin[['Postal Code', 'Borough', 'Neighborhood']]
df_edin

,Postal Code,Borough,Neighborhood
0,EH1,EDINBURGH,Mostly consists of Edinburgh's Old Town. Also ...
1,EH2,EDINBURGH,The New Town and central commercial area of Ed...
2,EH3,EDINBURGH,An odd shaped area surrounding EH1 and EH2 to ...
3,EH4,EDINBURGH,Radiates from the older and more central areas...
4,EH5,EDINBURGH,Based on a village formerly separate from the ...
5,EH6,EDINBURGH,"Covers Leith, as well as Newhaven bordering it..."
6,EH7,EDINBURGH,The inner city area between central Edinburgh ...
7,EH8,EDINBURGH,"The inner city Southside, Newington and Canong..."
8,EH9,EDINBURGH,"The inner city, Marchmont and Grange, Blackfor..."
9,EH10,EDINBURGH,"A corridor along the A702 from Bruntsfield, th..."


Lets get the Latitude and Longitude of each place.

In [85]:
Lat_list=[]
Lng_list=[]

for i in range(df_edin.shape[0]):
    address='{}, Edinburgh, Scotland'.format(df_edin.at[i,'Postal Code'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

In [86]:
df_edin['Latitude'] = Lat_list
df_edin['Longitude'] = Lng_list
df_edin[0:12]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,EH1,EDINBURGH,Mostly consists of Edinburgh's Old Town. Also ...,55.95309,-3.18899
1,EH2,EDINBURGH,The New Town and central commercial area of Ed...,55.95340,-3.20064
2,EH3,EDINBURGH,An odd shaped area surrounding EH1 and EH2 to ...,55.96030,-3.20309
3,EH4,EDINBURGH,Radiates from the older and more central areas...,55.96183,-3.26510
4,EH5,EDINBURGH,Based on a village formerly separate from the ...,55.97482,-3.21536
5,EH6,EDINBURGH,"Covers Leith, as well as Newhaven bordering it...",55.97465,-3.18367
6,EH7,EDINBURGH,The inner city area between central Edinburgh ...,55.95768,-3.17531
7,EH8,EDINBURGH,"The inner city Southside, Newington and Canong...",55.95507,-3.14836
8,EH9,EDINBURGH,"The inner city, Marchmont and Grange, Blackfor...",55.93524,-3.17574
9,EH10,EDINBURGH,"A corridor along the A702 from Bruntsfield, th...",55.90231,-3.20502


In [87]:
address = 'Edinburgh, Scotland'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Edinburgh are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Edinburgh are 55.9533456, -3.1883749.


In [89]:
# create map of Edinburgh using latitude and longitude values
map_edin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_edin['Latitude'], df_edin['Longitude'], df_edin['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3006cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edin)  
    
map_edin

Next step is to get 100 venues near each available place in our dataset by requesting the Foursquare API.

In [96]:
CLIENT_ID = 'EI1RT5DKEAPLTXBX3YWBFPYKX5O034BYAPI1TFZBCQWEEYVD' # your Foursquare ID
CLIENT_SECRET = 'UD1WQYEQKENGAIM3IZC0ALCAMFKBQPTB1AQOF0E5HVTERN0A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: EI1RT5DKEAPLTXBX3YWBFPYKX5O034BYAPI1TFZBCQWEEYVD
CLIENT_SECRET:UD1WQYEQKENGAIM3IZC0ALCAMFKBQPTB1AQOF0E5HVTERN0A


In [97]:
LIMIT = 500

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [132]:
edin_venues = getNearbyVenues(names=df_edin['Neighborhood'],
                                   latitudes=df_edin['Latitude'],
                                   longitudes=df_edin['Longitude']
                                  )

In [133]:
print(edin_venues.shape)
edin_venues.head()

(1109, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mostly consists of Edinburgh's Old Town. Also ...,55.95309,-3.18899,Dishoom,55.953726,-3.192540,Indian Restaurant
1,Mostly consists of Edinburgh's Old Town. Also ...,55.95309,-3.18899,The Balmoral Hotel,55.953113,-3.189509,Hotel
2,Mostly consists of Edinburgh's Old Town. Also ...,55.95309,-3.18899,SCOTCH Whisky Bar,55.953062,-3.190095,Whisky Bar
3,Mostly consists of Edinburgh's Old Town. Also ...,55.95309,-3.18899,Apple Princes Street,55.953354,-3.189947,Electronics Store
4,Mostly consists of Edinburgh's Old Town. Also ...,55.95309,-3.18899,The Milkman,55.950650,-3.191010,Coffee Shop


In [134]:
edin_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A corridor (rather thin in shape) along the A71 from Haymarket Station, through Gorgie and Stenhouse to Sighthill and the Calders.",26,26,26,26,26,26
"A corridor along the A702 from Bruntsfield, through Morningside as far as Fairmilehead.",5,5,5,5,5,5
A corridor along the A8 from Haymarket through Murrayfield and Corstorphine as far as the Gyle.,21,21,21,21,21,21
"A corridor in south-west Edinburgh starting at Slateford, through Longstone, Wester Hailes, Juniper Green, Currie and on to Balerno.",26,26,26,26,26,26
"An odd shaped area surrounding EH1 and EH2 to the north, west and south. It can be divided into three distinct areas: The New Town area, north of Queen Street and northwards to Stockbridge. The West End, including some of Edinburgh's financial district The Tollcross and Fountainbridge areas.",100,100,100,100,100,100
"Based on Portobello and Duddingston, formerly separate settlements to Edinburgh.",22,22,22,22,22,22
"Based on a village formerly separate from the city; in this case, Granton on the Firth of Forth. EH5 extends inwards from the coast to Ferry Road.",18,18,18,18,18,18
"Based on the formerly separate village of Gilmerton, taking in Moredun and extending westwards as far as Mortonhall.",7,7,7,7,7,7
"Based on the formerly separate village of Liberton. It extends north and north-east to Cameron Toll, Craigmillar and Niddrie.",16,16,16,16,16,16


In [135]:
print('There are {} uniques categories.'.format(len(edin_venues['Venue Category'].unique())))

There are 173 uniques categories.


In [136]:
# one hot encoding
edin_onehot = pd.get_dummies(edin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
edin_onehot['Neighborhood'] = edin_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [edin_onehot.columns[-1]] + list(edin_onehot.columns[:-1])
edin_onehot = edin_onehot[fixed_columns]

edin_grouped = edin_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in edin_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = edin_grouped[edin_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A corridor (rather thin in shape) along the A71 from Haymarket Station, through Gorgie and Stenhouse to Sighthill and the Calders.----
            venue  freq
0     Supermarket  0.12
1   Grocery Store  0.08
2  Discount Store  0.04
3     Coffee Shop  0.04
4            Park  0.04


----A corridor along the A702 from Bruntsfield, through Morningside as far as Fairmilehead.----
                      venue  freq
0  Mediterranean Restaurant   0.2
1         Convenience Store   0.2
2               Pizza Place   0.2
3                       Pub   0.2
4      Outdoor Supply Store   0.2


----A corridor along the A8 from Haymarket through Murrayfield and Corstorphine as far as the Gyle.----
                venue  freq
0       Grocery Store  0.19
1                 Pub  0.10
2   Indian Restaurant  0.05
3         Supermarket  0.05
4  Italian Restaurant  0.05


----A corridor in south-west Edinburgh starting at Slateford, through Longstone, Wester Hailes, Juniper Green, Currie and on to Balerno.---

In [137]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = edin_grouped['Neighborhood']

for ind in np.arange(edin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(edin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,A corridor (rather thin in shape) along the A7...,Supermarket,Grocery Store,Fish & Chips Shop,Gas Station,Coffee Shop
1,"A corridor along the A702 from Bruntsfield, th...",Convenience Store,Pub,Outdoor Supply Store,Mediterranean Restaurant,Pizza Place
2,A corridor along the A8 from Haymarket through...,Grocery Store,Pub,Gym / Fitness Center,Supermarket,Chinese Restaurant
3,A corridor in south-west Edinburgh starting at...,Supermarket,Coffee Shop,Grocery Store,Gas Station,Tennis Court
4,An odd shaped area surrounding EH1 and EH2 to ...,Café,Bar,Pub,Coffee Shop,Bakery


In [138]:
# set number of clusters
kclusters = 8

edin_grouped_clustering = edin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(edin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([5, 6, 5, 5, 0, 0, 0, 1, 0, 3, 0, 0, 0, 4, 7, 0, 0, 2, 0, 0, 0, 1,
       0, 0], dtype=int32)

In [139]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

edin_merged = df_edin

# merge edin_grouped with edin_data to add latitude/longitude for each neighborhood
edin_merged = edin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [140]:
number_clusters = len(edin_merged['Cluster Labels'].unique())
print('There are {} clusters.'.format(number_clusters))

There are 8 clusters.


In [141]:
edin_merged = edin_merged[edin_merged['Cluster Labels'].notna()]

In [142]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(edin_merged['Latitude'], edin_merged['Longitude'], edin_merged['Neighborhood'], edin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Let's check the number of neighborhood per cluster

In [143]:
for i in range(number_clusters):
    print("Cluster n {} has {} group of neighborhoods".format(i, len(edin_merged.loc[edin_merged['Cluster Labels'] == i])))

Cluster n 0 has 14 group of neighborhoods
Cluster n 1 has 2 group of neighborhoods
Cluster n 2 has 1 group of neighborhoods
Cluster n 3 has 1 group of neighborhoods
Cluster n 4 has 1 group of neighborhoods
Cluster n 5 has 3 group of neighborhoods
Cluster n 6 has 1 group of neighborhoods
Cluster n 7 has 1 group of neighborhoods


In [144]:
edin_merged.loc[edin_merged['Cluster Labels'] == 0, edin_merged.columns[[1] + list(range(6, edin_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,EDINBURGH,Café,Hotel,Restaurant,Pub,Coffee Shop
1,EDINBURGH,Café,Bar,Cocktail Bar,Restaurant,Whisky Bar
2,EDINBURGH,Café,Bar,Pub,Coffee Shop,Bakery
4,EDINBURGH,Rugby Pitch,Park,Garden,Pub,Stadium
5,EDINBURGH,Bar,Café,Pub,Seafood Restaurant,Supermarket
6,EDINBURGH,Bar,Café,Bakery,Grocery Store,Pub
8,EDINBURGH,Pub,Hotel,Restaurant,Bar,Indian Restaurant
14,EDINBURGH,Furniture / Home Store,Bakery,Pub,Park,Carpet Store
15,EDINBURGH,Hotel,Fast Food Restaurant,Clothing Store,Athletics & Sports,Coffee Shop
18,KIRKLISTON,Hotel,Coffee Shop,Pub,Grocery Store,Café


In [145]:
edin_merged.loc[edin_merged['Cluster Labels'] == 1, edin_merged.columns[[1] + list(range(6, edin_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,EDINBURGH,Grocery Store,Fish & Chips Shop,Supermarket,Park,Bakery
16,EDINBURGH,Grocery Store,Supermarket,Fast Food Restaurant,Park,Pizza Place


In [146]:
edin_merged.loc[edin_merged['Cluster Labels'] == 2, edin_merged.columns[[1] + list(range(6, edin_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,EDINBURGH,Park,Museum,Gym / Fitness Center,Campground,Bakery


In [147]:
glasgow_merged.loc[edin_merged['Cluster Labels'] == 3, edin_merged.columns[[1] + list(range(6, edin_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,EDINBURGH,Trail,Coffee Shop,Supermarket,Beer Garden,Park


In [148]:
edin_merged.loc[edin_merged['Cluster Labels'] == 4, edin_merged.columns[[1] + list(range(6, edin_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,NEWBRIDGE,Café,Boat or Ferry,Gym / Fitness Center,Playground,Wings Joint


In [149]:
edin_merged.loc[edin_merged['Cluster Labels'] == 5, edin_merged.columns[[1] + list(range(6, edin_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,EDINBURGH,Supermarket,Grocery Store,Fish & Chips Shop,Gas Station,Coffee Shop
11,EDINBURGH,Grocery Store,Pub,Gym / Fitness Center,Supermarket,Chinese Restaurant
13,"BALERNO, CURRIE, EDINBURGH, JUNIPER GREEN",Supermarket,Coffee Shop,Grocery Store,Gas Station,Tennis Court


In [150]:
edin_merged.loc[edin_merged['Cluster Labels'] == 6, edin_merged.columns[[1] + list(range(6, edin_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,EDINBURGH,Convenience Store,Pub,Outdoor Supply Store,Mediterranean Restaurant,Pizza Place
